In [13]:
import geopandas as gpd
import pandas as pd
import glob
from shapely.geometry import MultiPoint, MultiLineString, LineString, Point
from shapely.ops import split, unary_union, snap, linemerge

In [3]:
distritos = gpd.read_file(f'downloads/SIRGAS_GPKG_distrito.gpkg')

In [16]:
for i, d in distritos.iterrows():
    folder = f'{d.ds_codigo} - {d.ds_nome}'
    print(folder)
    
    faixas_poly = glob.glob(f'resultados/vetor/{folder}/faixas-poly-*.gpkg')
    gdf_faixas_poly = gpd.GeoDataFrame(pd.concat([gpd.read_file(f) for f in faixas_poly]))
    gdf_faixas_poly = gdf_faixas_poly.set_crs(epsg=31983)

    linhas_de_travessia = glob.glob(f'resultados/vetor/{folder}/linhas-de-travessia-*.gpkg')
    gdf_linhas_de_travessia = gpd.GeoDataFrame(pd.concat([gpd.read_file(f) for f in linhas_de_travessia]))
    gdf_linhas_de_travessia = gdf_linhas_de_travessia.set_crs(epsg=31983)

    gdf_rotas_mediais = gpd.read_file(f'../rotas-em-calcadas/resultados/eixos_mediais/{d.ds_codigo}-eixos-mediais-{d.ds_nome}.gpkg')
    gdf_calcadas = gpd.read_file(f'../rotas-em-calcadas/resultados/calcadas/{d.ds_codigo}-calcadas-{d.ds_nome}.gpkg')

    rotas_caminhaveis = gdf_linhas_de_travessia.unary_union.union(gdf_rotas_mediais.unary_union)
    geometrias = [i for i in linemerge(rotas_caminhaveis)]

    gdf_rotas_caminhaveis = gpd.GeoDataFrame(geometry=geometrias).set_crs(epsg=31983)
    gdf_rotas_caminhaveis.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
    driver='GPKG', layer='rede_caminhavel')
    gdf_faixas_poly.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
    driver='GPKG', layer='faixas_poly')
    gdf_linhas_de_travessia.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
    driver='GPKG', layer='linhas_de_travessia')
    gdf_rotas_mediais.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
    driver='GPKG', layer='rotas_mediais')
    gdf_calcadas.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
    driver='GPKG', layer='calcadas')
    # [print(g.crs) for g in [gdf_faixas_poly, gdf_linhas_de_travessia, gdf_rotas_mediais, gdf_calcadas]]

    break

63 - PIRITUBA


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate 